# Unemployment vs Inflation
## Loading the data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/inflation-and-unemployment/data.csv')

In [ ]:
df.tail(10)

## Data Cleaning

In [ ]:
df.info()

In [ ]:
import re

def clean_years(years):
    return [int(re.sub(r" \[\w+\]$", "", year)) for year in years]

In [ ]:
years = df.columns[4:]
years = clean_years(years)
df.columns = list(df.columns[:4]) + years
df.columns

In [ ]:
df[years] = df[years].apply(pd.to_numeric, errors='coerce')
df.info()

In [ ]:
def get_missing_data(data):
    missing_data = data.isna().sum()
    proportions = (missing_data / len(data)
                               ).sort_values(ascending=False)
    return missing_data, proportions

In [ ]:
missing_data, proportions = get_missing_data(df)
proportions

In [ ]:
df.tail(10)

In [ ]:
df.drop(df.tail().index, inplace=True)

missing_data, missing_data_proportions = get_missing_data(df)
missing_data_proportions

In [ ]:
countries = df['Country Name'].unique()
print(f'Countries: {countries}')

In [ ]:
series = df['Series Name'].unique()
print(f'Series: {series}')

In [ ]:
def list_search(iterable, search_term):
    results = [x for x in iterable 
               if re.search(search_term, x, re.IGNORECASE)]
    return results

In [ ]:
inflation = list_search(series, 'GDP deflator')
unemployment =  list_search(series, 'unemployment, total')
inflation_unemployment = inflation + unemployment
print(inflation_unemployment)

In [ ]:
filtered_df = df[df['Series Name'].isin(inflation_unemployment)]
filtered_df.head()

In [ ]:
data = filtered_df[years].T
iterables = [countries, inflation_unemployment]
columns = pd.MultiIndex.from_product(iterables, names=['country', 'series'])

dataframe = pd.DataFrame(data.values, index=data.index, columns=columns)
dataframe.tail()

In [ ]:
missing_data, proportions = get_missing_data(dataframe)

# select series with missing data proportion that is less than a threshold
threshold = 0.05
less_missing_data = proportions[proportions < threshold]
less_missing_data

## Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (14,8)
sns.set()

In [ ]:
def clean_series(series_name):
    return series_name.split(' (')[0]

In [ ]:
def lineplot(data, country):
    fig, ax = plt.subplots()
    data = data[country]

    sns.lineplot(data=data, ax=ax)
    series_names = data.columns
    ax.set_title("""Line plots of inflation and unemployment rates for {}
        from {} to {}""".format(
        country, data.index.min(), data.index.max()
    ))
    ax.set_xlabel('Years')
    ax.set_ylabel('% rates')
    plt.show()

def regplot(data, country, x=None, y=None):
    fig, ax = plt.subplots()
    data = data[country]
    
    sns.regplot(x=x, y=y, data=data, ax=ax)
    ax.set_title("""Scatterplot of {} vs {} for {} 
        from {} to {}""".format(
        clean_series(x), clean_series(y), country,
        data.index.min(), data.index.max()
    ))
    plt.show()

In [ ]:
unemployment_column = inflation_unemployment[3]
inflation_column = inflation_unemployment[0]

In [ ]:
countries_list = [
    'Austria', 'Belgium', 'China', 'Costa Rica', 'Cyprus',
    'Czech Republic', 'Finland', 'France', 'Germany', 'Greece',
    'Hong Kong SAR, China', 'Japan', 'Macao SAR, China',
    'Norway', 'Panama', 'Portugal', 'Serbia', 'Singapore',
    'Spain', 'Switzerland', 'Turkey', 'West Bank and Gaza',
    'United Kingdom', 'United States',
]

In [ ]:
inflation_change_column = 'Change in inflation'

iterables = [countries, [inflation_column]]
inflation_cols = pd.MultiIndex.from_product(iterables, names=['country', 'series'])

iterables = [countries, [inflation_change_column]]
inflation_change_cols = pd.MultiIndex.from_product(iterables, names=['country', 'series'])

inflation_change = dataframe[columns]
inflation_change[inflation_change_cols] = inflation_change[inflation_cols].diff()
inflation_change.head()

In [ ]:
iterables = [
    [countries_list[0]],
    [inflation_column, inflation_change_column]
]
country_cols = pd.MultiIndex.from_product(iterables, names=['country', 'series'])
inflation_change[country_cols].head(10)

In [ ]:
for country in countries:
    # time series
    lineplot(inflation_change, country)
    # Phillip's curve
    regplot(inflation_change, country, x=unemployment_column,
            y=inflation_column)
    # Phillip's curve with adaptive expectations
    regplot(inflation_change, country,
            x=inflation_change_column,
            y=unemployment_column)

In [ ]:
# sample_countries = ['France', 'United Kingdom', 'United States']

# for country in sample_countries:
#     iterables = [
#         [country],
#         [inflation_column, inflation_change_column, unemployment_column]
#     ]
#     country_cols = pd.MultiIndex.from_product(iterables, names=['country', 'series'])
#     data = inflation_change[country_cols]
#     display(data.tail(30))
#     regplot(data, country, x=inflation_change_column, y=unemployment_column)